In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os
pyspark.__version__

'3.0.0'

In [2]:
sc = SparkContext("local[*]","PySpark Tutorial")  # Use all cores

In [3]:
spark = SparkSession.builder.appName('lda_modeling').getOrCreate()

In [ ]:
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

In [10]:
#data = sqlContext.read.format("csv").options(header='true', inferschema='true').load(os.path.realpath("us_equities_news_dataset.csv"))
data = spark.read.options(header='True', inferschema='True').csv(os.path.realpath("us_equities_news_dataset.csv"))

In [11]:
type(data['content'])

pyspark.sql.column.Column

In [13]:
news_content = data.rdd.map(lambda x : x['content']).filter(lambda x: x is not None)

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words += ['tesla', 'tsla', 'said']

In [18]:
import re as re
tokens = news_content                                                   \
    .map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if x not in stop_words])    \
    .zipWithIndex()

In [19]:
type(tokens)

pyspark.rdd.PipelinedRDD

In [21]:
text_df = spark.createDataFrame(tokens, ['words','index'])

In [48]:
from pyspark.ml.feature import CountVectorizer , IDF
cv = CountVectorizer(inputCol="words", outputCol="raw_features", vocabSize=5000, minDF=10.0)
cvmodel = cv.fit(text_df)
result_cv = cvmodel.transform(text_df)

idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv)

In [49]:
from pyspark.ml.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA, LDAModel
lda = LDA(k=19, maxIter=20)
#result_df = spark.createDataFrame(result_tfidf, ['index','features'])
lda_model = lda.fit(result_tfidf[['index','features']])

In [50]:
topics = lda_model.describeTopics(19)
topics.show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                                                                               |termWeights                                                                                                                                                                                                                                                                                                                                            

In [56]:
transformed = lda_model.transform(result_tfidf[['index','features']])

In [57]:
transformed.show(1, truncate=False)

+-----+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|index|features                         |topicDistribution                                                                                                                                                                                                                                                                                                                                                                                                |
+-----+---------------------------------+-----------------------------------------------------------------------

In [58]:
transformed.show(2)

+-----+--------------------+--------------------+
|index|            features|   topicDistribution|
+-----+--------------------+--------------------+
|    0|(5000,[3689],[5.6...|[0.86662234634245...|
|    1|        (5000,[],[])|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+--------------------+
only showing top 2 rows



In [62]:
import pandas as pd
pd_df = transformed.toPandas()

In [69]:
pd_df['features'].iloc[0]

SparseVector(5000, {3689: 5.6156})